In [3]:
import sys
import numpy as np
import lightgbm as lg
from tqdm import tqdm

sys.path.append("../../scorer/")
# import orderbook as ob
# Чтобы использовать быстрый ордербук раскомментируйте строку:
import orderbook_fast as ob
 
SIDE_BID = 0 
SIDE_ASK = 1

/home/sergey/anaconda3/envs/wunder_summer/lib/python3.8/site-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /home/sergey/mnt/st1500/Usr/Sergey/TheJob/Challenges/wunder_summer/wunder_challenge/scorer/orderbook_fast/orderbook_fast.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


## Собираем датасет для тренировки модели

In [4]:
def get_simple_features_from_orderbook(orderbook, max_index=2):
    '''
        Считаем простые фичи по ордербуку:
    '''
    spread = orderbook.get_best_price(SIDE_ASK) - orderbook.get_best_price(SIDE_BID)
    features = [spread]
    for side in (SIDE_BID, SIDE_ASK):
        for ix in range(max_index):
            price_level = orderbook.get_price_level_at_ix(side, ix)
            if price_level is None:
                features += [-1, -1]
            else:
                features += [price_level.get_volume(), 
                             price_level.get_num_orders()]
    return features


def get_simple_deals_features(last_deals, orderbook):
    '''
        Считаем простые фичи по последним сделкам:
    '''
    cur_mean_price = orderbook.get_mean_price()
    cur_time = orderbook.get_time()

    features = []
    for side in (SIDE_BID, SIDE_ASK):
        deal_event = last_deals[side]
        if deal_event is None:
            features += [-1e9, -1e9, -1e9]
        else:
            features += [cur_mean_price - deal_event.price, 
                         cur_time - deal_event.time, 
                         deal_event.amount]
    return features


def collect_dataset(data_path):
    '''
        Собираем датасет
    '''
    event_player = ob.EventPlayer(data_path)
    orderbook = ob.OrderBook()

    X = []
    Y = []

    last_deals = [None, None]
    for ev in tqdm(event_player.iget_events(), 
                    total=len(event_player), 
                    desc="collecting dataset"):
        if ev.action == ob.Action.DEAL:
            last_deals[ev.side] = ev
        elif ev.action == ob.Action.NEW_CHUNK:
            last_deals = [None, None]
        
        orderbook.apply_event(ev)
        if ev.need_prediction:
            features = get_simple_features_from_orderbook(orderbook)
            features += get_simple_deals_features(last_deals, orderbook)

            X.append(features)
            Y.append(ev.Y)

    print(f"Dataset collected: len(X) = {len(X)}")
    return np.array(X), np.array(Y)


X_train, Y_train = collect_dataset("../../data/train_small_A.npz")
X_test, Y_test = collect_dataset("../../data/train_small_B.npz")

collecting dataset: 100%|██████████| 10555835/10555835 [01:04<00:00, 163449.01it/s]


Dataset collected: len(X) = 234905
Dataset collected: len(X) = 240309


## Обучаем модель градиентного бустинга

In [5]:
def train_classifier(X_train, Y_train, X_test, Y_test):
    '''
        Обучаем классификатор
    '''
    clf = lg.LGBMClassifier(num_leaves=31, n_estimators=1000, learning_rate=0.1)
    clf.fit(X_train, Y_train, eval_set=[(X_test, Y_test)], 
            eval_metric="auc", early_stopping_rounds=20)
    return clf
 
clf = train_classifier(X_train, Y_train, X_test, Y_test)

[1]	valid_0's auc: 0.62018	valid_0's binary_logloss: 0.481247
Training until validation scores don't improve for 20 rounds
[2]	valid_0's auc: 0.62531	valid_0's binary_logloss: 0.479038
[3]	valid_0's auc: 0.626034	valid_0's binary_logloss: 0.477295
[4]	valid_0's auc: 0.626741	valid_0's binary_logloss: 0.475821
[5]	valid_0's auc: 0.627089	valid_0's binary_logloss: 0.474582
[6]	valid_0's auc: 0.627325	valid_0's binary_logloss: 0.473539
[7]	valid_0's auc: 0.628326	valid_0's binary_logloss: 0.472596
[8]	valid_0's auc: 0.628691	valid_0's binary_logloss: 0.471835
[9]	valid_0's auc: 0.629272	valid_0's binary_logloss: 0.47112
[10]	valid_0's auc: 0.629673	valid_0's binary_logloss: 0.470545
[11]	valid_0's auc: 0.630045	valid_0's binary_logloss: 0.470049
[12]	valid_0's auc: 0.630495	valid_0's binary_logloss: 0.469601
[13]	valid_0's auc: 0.630722	valid_0's binary_logloss: 0.469228
[14]	valid_0's auc: 0.631064	valid_0's binary_logloss: 0.468878
[15]	valid_0's auc: 0.631368	valid_0's binary_logloss: 

## Тестируем получившийся классификатор

In [6]:
last_deals = [None, None]

def process_event_and_predict_proba(event, orderbook):
    if event.action == ob.Action.DEAL:
        last_deals[event.side] = event
    elif event.action == ob.Action.NEW_CHUNK:
        last_deals[:] = [None, None]
        
    if not event.need_prediction:
        return None
    
    features = get_simple_features_from_orderbook(orderbook)
    features += get_simple_deals_features(last_deals, orderbook)    
    proba = clf.predict_proba([features])[0, 1]
    return proba

from scorer import Scorer

scoring = Scorer("../../data/train_small_C.npz")
roc_auc, (true_ys, pred_probas) = scoring.score(process_event_and_predict_proba)

scoring: 100%|██████████| 10562435/10562435 [06:37<00:00, 26586.74it/s]



roc_auc_score = 0.636


## Сохраним модель, и решение для отправки готово

In [7]:
# Сохраним нашу модель
clf.booster_.save_model("wunder.model")

'''Посмотрите код файла solution.py. 
Он использует те же функции что и этот ноутбук, но уже готов к отправке на серверю. 
Попробуйте создать архив с файлами solution.py и wunder.model и отправить их на проверку.'''

'Посмотрите код файла solution.py. \nОн использует те же функции что и этот ноутбук, но уже готов к отправке на серверю. \nПопробуйте создать архив с файлами solution.py и wunder.model и отправить их на проверку.'